In [2]:
import pandas as pd
import numpy as np
import random
from tqdm import tqdm
from gensim.models import Word2Vec 
import matplotlib.pyplot as plt
%matplotlib inline

import warnings;
warnings.filterwarnings('ignore')

In [4]:
df = pd.read_excel('/home/jeovine/Desktop/ML/Retail Online/Online Retail (2).xlsx')

In [6]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [8]:
df.to_csv('Online Retail.csv')

In [9]:
df = pd.read_csv('/home/jeovine/Desktop/ML/Retail Online/Online Retail.csv')

In [10]:
df.head()

,Unnamed: 0,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [11]:
df.describe()

,Unnamed: 0,Quantity,UnitPrice,CustomerID
count,541909.00000,541909.000000,541909.000000,406829.000000
mean,270954.00000,9.552250,4.611114,15287.690570
std,156435.79785,218.081158,96.759853,1713.600303
min,0.00000,-80995.000000,-11062.060000,12346.000000
25%,135477.00000,1.000000,1.250000,13953.000000
50%,270954.00000,3.000000,2.080000,15152.000000
75%,406431.00000,10.000000,4.130000,16791.000000
max,541908.00000,80995.000000,38970.000000,18287.000000


In [14]:
df.shape

(541909, 9)

In [15]:
df.isnull().sum()

Unnamed: 0          0
InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [16]:
df.dropna(inplace=True)

In [17]:
df.isnull().sum()

Unnamed: 0     0
InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

 * Let’s convert the StockCode to string datatype:

In [19]:
df['StockCode']= df['StockCode'].astype(str)

 * checking out the number of unique customers in our dataset:

In [24]:
customers = df["CustomerID"].unique().tolist()

* There are 4,372 customers in our dataset.
* For each of these customers, we will extract their buying history. 
* In other words, we can have 4,372 sequences of purchases.

* It is a good practice to set aside a small part of the dataset for validation purposes. 
* Therefore, I will use the data of 90% of the customers to create word2vec embeddings.
* Let’s split the data.

In [26]:
# shuffle customer ID's
random.shuffle(customers)

# extract 90% of customer ID's
customers_train = [customers[i] for i in range(round(0.9*len(customers)))]

# split data into train and validation set
train_df = df[df['CustomerID'].isin(customers_train)]
validation_df = df[~df['CustomerID'].isin(customers_train)]

* We will create sequences of purchases made by the customers in the dataset for both the train and validation set.

In [27]:
# list to capture purchase history of the customers
purchases_train = []

# populate the list with the product codes
for i in tqdm(customers_train):
    temp = train_df[train_df["CustomerID"] == i]["StockCode"].tolist()
    purchases_train.append(temp)

100%|██████████| 3935/3935 [00:08<00:00, 451.85it/s]


In [28]:
# list to capture purchase history of the customers
purchases_val = []

# populate the list with the product codes
for i in tqdm(validation_df['CustomerID'].unique()):
    temp = validation_df[validation_df["CustomerID"] == i]["StockCode"].tolist()
    purchases_val.append(temp)

100%|██████████| 437/437 [00:00<00:00, 463.52it/s]


# A Build for word2vec Embeddings on Products

In [29]:
# train word2vec model
model = Word2Vec(window = 10, sg = 1, hs = 0,
                 negative = 10, # for negative sampling
                 alpha=0.03, min_alpha=0.0007,
                 seed = 14)

model.build_vocab(purchases_train, progress_per=200)

model.train(purchases_train, total_examples = model.corpus_count, 
            epochs=10, report_delay=1)


(3585686, 3623230)

* Since we are not planning to train the model any further, we are calling init_sims( ) here. 
* Thus will make the model much more memory-efficient:

In [30]:
model.init_sims(replace=True)


# Checking out the summary of “model”:


In [32]:
print(model)


Word2Vec(vocab=3155, size=100, alpha=0.03)


* Our model has a vocabulary of 3,155 unique words and their vectors of size 100 each. 
* Next, we will extract the vectors of all the words in our vocabulary and store it in one place for easy access.

In [33]:
# extract all vectors
X = model[model.wv.vocab]

X.shape

(3155, 100)

# Visualize word2vec Embeddings
It is always quite helpful to visualize the embeddings that you have created. Over here, we have 100-dimensional embeddings. We can’t even visualize 4 dimensions let alone 100. 
Hence we reduce the dimensions of the product embeddings from 100 to 2 by using the UMAP algorithm. It is popularly used for dimensionality reduction.

# Recommending Products
We are finally ready with the word2vec embeddings for every product in our online retail dataset. Now, our next step is to suggest similar products for a certain product or a product’s vector.

Lets first start by creating the product-ID and product-description dictionary to easily map a product’s description to its ID and vice versa.



In [39]:
products = train_df[["StockCode", "Description"]]

# remove duplicates
products.drop_duplicates(inplace=True, subset='StockCode', keep="last")

# create product-ID and product-description dictionary
products_dict = products.groupby('StockCode')['Description'].apply(list).to_dict()

# Testing the Dictionary

In [40]:
# test the dictionary 
products_dict['84029E']

['RED WOOLLY HOTTIE WHITE HEART.']

* The Function Below will take a product’s vector (n) as input and return top 6 similar products:

In [41]:
def similar_products(v, n = 6):
    
    # extract most similar products for the input vector
    ms = model.similar_by_vector(v, topn= n+1)[1:]
    
    # extract name and similarity score of the similar products
    new_ms = []
    for j in ms:
        pair = (products_dict[j[0]][0], j[1])
        new_ms.append(pair)
        
    return new_ms   

# Testing Above Function
Let’s try out our function by passing the vector of the product ‘90019A’ 

In [42]:
similar_products(model['90019A'])


[('SILVER M.O.P ORBIT DROP EARRINGS', 0.765789270401001),
 ('RASPBERRY ANT COPPER FLOWER NECKLAC', 0.7531300783157349),
 ('AMBER DROP EARRINGS W LONG BEADS', 0.7520068883895874),
 ('PINK BOUDICCA LARGE BRACELET', 0.7488296627998352),
 ('ANT COPPER RED BOUDICCA BRACELET', 0.7456411123275757),
 ('JADE DROP EARRINGS W FILIGREE', 0.7413666248321533)]

* Cool! i've got big huevos the results are pretty perfect.

# Getting predictions/output for multiple purchases
However, this output is based on the vector of a single product only. What if we want to recommend products based on the multiple purchases he or she has made in the past?

One easy simple solution is to take the average of all the vectors of the products the user has bought so far and use this resultant vector to find similar products. We will use the function below that takes in a list of product IDs and gives out a 100-dimensional vector which is a mean of vectors of the products in the input list:

In [44]:
def aggregate_vectors(products):
    product_vec = []
    for i in products:
        try:
            product_vec.append(model[i])
        except KeyError:
            continue
        
    return np.mean(product_vec, axis=0)


* Recall that we had already created a separate list of purchase sequences for validation purposes. 
* Now let’s make use of that.


In [45]:
len(purchases_val[0])


17

# Output: 17
The length of the first list of products purchased by a user is 17.
We will pass this products’ sequence of the validation set to the function aggregate_vectors.



In [46]:
aggregate_vectors(purchases_val[0]).shape


(100,)

# Output: (100, )

Well, the function has returned an array of 100 dimensions. It means the function is working fine. Now we can use this result to get the most similar products:



In [48]:
similar_products(aggregate_vectors(purchases_val[0]))


[('RIBBON REEL HEARTS DESIGN ', 0.7149956822395325),
 ('RIBBON REEL FLORA + FAUNA ', 0.7081438302993774),
 ('RIBBON REEL LACE DESIGN ', 0.6810587644577026),
 ('PACK OF 6 BIRDY GIFT TAGS', 0.6631482839584351),
 ('6 RIBBONS RUSTIC CHARM', 0.6336013674736023),
 ('PLASTERS IN TIN VINTAGE PAISLEY ', 0.6329262256622314)]

* As it turns out, our system has recommended 6 products based on the entire purchase history of a user. 
* Moreover, if you want to get product suggestions based on the last few purchases, only then also you can use the same set of functions.


# Below I am giving only the last 6 products purchased as input:


In [52]:
similar_products(aggregate_vectors(purchases_val[0][-6:]))


[('RIBBON REEL HEARTS DESIGN ', 0.7359219789505005),
 ('RIBBON REEL FLORA + FAUNA ', 0.7281462550163269),
 ('IVORY GIANT GARDEN THERMOMETER', 0.6925832033157349),
 ('RIBBON REEL LACE DESIGN ', 0.6881155967712402),
 ('6 RIBBONS RUSTIC CHARM', 0.6452010869979858),
 ('PACK OF 6 BIRDY GIFT TAGS', 0.631314754486084)]